## Compute regional vaccine dosage needed to achieve a given coverage in 3 months
- target coverage to be input below

In [1]:
from ABMpy_region.model import Meta_ABM 
from ABMpy_region.agent import ABM
import numpy as np
import pickle
import copy
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict 
import os
import sys
import matplotlib.backends.backend_pdf

## scenario 1

every country 10% by end of Sep

In [146]:
# read in baseline projection data 
meta={}
file='vaccine_scenarios_712/meta_'+'base'+'_data0'+'.pkl'

with open(file,'rb') as f:
    meta['base']=(pickle.load(f))
regions=[]             
for i in range(198):
    r=meta['base'][i]['region']
    regions.append(r)

In [77]:
# vaccine dosage file 
vaccine=pd.read_csv("vaccine_info_83.csv")
vaccine.drop(columns='govex',inplace=True)



In [78]:
# only regions we simulated 
vaccine=vaccine[vaccine.location.isin(regions)]

In [79]:
for i in range(198):
    r=meta['base'][i]['region']
    vaccine.loc[vaccine.location==r,'population']=meta['base'][i]['actual_pop_scale']*10**6


In [80]:
# impute NULL with 0
vaccine.loc[vaccine['dailyVac1_permillion_duringLastMonth'].isnull(),'dailyVac1_permillion_duringLastMonth']=0

In [85]:
# vaccine coverage as of 6/30/2021 
vaccine['vacRate_546(6/30)']=vaccine['vac1RateCumu_asOfLastMonth']*2+100*vaccine.dailyVac1_permillion_duringLastMonth*2*30/10**6

In [86]:
# baseline vaccine coverage 9/30
vaccine['vacRate_9/30']=(vaccine['vacRate_546(6/30)']+100*(92*vaccine.dailyVac1_permillion_duringLastMonth*2/10**6))
vaccine['vacRate_9/30']=vaccine['vacRate_9/30'].clip(upper=100)

In [14]:
#vaccine

### INPUT HERE

In [124]:
target_percentage=26.012 #26.059 

# if baseline coverage on 9/30 < target: add donation
for i,row in vaccine.iterrows():
    r=row.location
    if row['vacRate_9/30']<target_percentage:   
        vaccine.loc[i,'new_dailyvac1Rate_to_6/30-9/30_10%']= (target_percentage-row['vacRate_546(6/30)'])/100*10**6/92/2
    else:
        vaccine.loc[i,'new_dailyvac1Rate_to_6/30-9/30_10%']=row.dailyVac1_permillion_duringLastMonth

In [125]:
# additional global vaccine needed 
vaccine['donation']=(vaccine['new_dailyvac1Rate_to_6/30-9/30_10%']-vaccine['dailyVac1_permillion_duringLastMonth'])*2*vaccine.population/10**6
total_donation=vaccine['donation'].sum()
print(total_donation)


5000247.29804556


total_donation needs to be recorded 

In [126]:
# baseline given total_donation
'''compute the distribution of vaccines if donation distributes by actual supply'''

# actual dosage supply last month, scaled back to population szie
vaccine['self_actual_dose']=vaccine['dailyVac1_permillion_duringLastMonth']*2*vaccine['population']/10**6
# absolute value of baseline donation
vaccine['baseline_donation(abs)']=total_donation*vaccine['self_actual_dose']/vaccine['self_actual_dose'].sum()
# scaled to dosage per million
vaccine['baseline_dosage_permillion_vac1']=vaccine['baseline_donation(abs)']/vaccine['population']*10**6/2+vaccine['dailyVac1_permillion_duringLastMonth']

In [127]:
# sanity check, the two distribution should give the same total donation 
((vaccine.baseline_dosage_permillion_vac1-vaccine.dailyVac1_permillion_duringLastMonth)*2*vaccine.population/10**6).sum()

5000247.298045562

In [128]:
vaccine.to_csv("vaccine_info_83_"+str(int(target_percentage))+"percent_plan.csv",index=False)